In [381]:
import numpy as np
import tensorflow as tf
import pickle as pkl
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional

In [442]:
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"
with open(datapath+"X_Y", "rb") as f:
    X, Y = pkl.load(f)
with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "rb") as f:
    vocab = pkl.load(f)


In [441]:
with open(datapath+"Train", "rb") as f:
    X_train, X_test = pkl.load(f)
with open(datapath+"Test", "rb") as f:
    X_test, Y_test = pkl.load(f)
with open(datapath+"Eval", "rb") as f:
    X_eval, Y_eval = pkl.load(f)

In [443]:
vocab_size=len(vocab)
RELATION_VOCAB_SIZE=27

In [400]:
def CELoss(y_pred, x):
    on_stroke_indicator=x[:,:,6]
    #assumes that y_pred is log probabilities
    #log_prob_of_symbol = tf.math.reduce_logsumexp(y_pred[:,:,RELATION_VOCAB_SIZE:], axis=2)
    prob_of_symbol = tf.math.reduce_sum(y_pred[:,:,RELATION_VOCAB_SIZE:], axis=2)
    return(tf.math.reduce_sum(on_stroke_indicator*(-tf.math.log(prob_of_symbol))))

In [13]:
y_offstroke_vector=np.zeros(vocab_size)
y_offstroke_vector[RELATION_VOCAB_SIZE:]=0.1/(vocab_size-RELATION_VOCAB_SIZE)
y_offstroke_vector[:RELATION_VOCAB_SIZE]=0.9/RELATION_VOCAB_SIZE
y_onstroke_vector=np.zeros(vocab_size)
y_onstroke_vector[RELATION_VOCAB_SIZE:]=0.9/(vocab_size-RELATION_VOCAB_SIZE)
y_onstroke_vector[:RELATION_VOCAB_SIZE]=0.1/RELATION_VOCAB_SIZE

def get_y_pred(x, y_offstroke_vector, y_onstroke_vector):
    #x input shape = (batch_size, sequence_len, 7)
    y = np.zeros((x.shape[0], x.shape[1], y_offstroke_vector.shape[0]))
    y=np.multiply.outer(x[:,:,6], y_onstroke_vector)+np.multiply.outer(1-x[:,:,6], y_offstroke_vector)
    
    #y[x[:,:,6] == 1] =  y_onstroke_vector
    #print(x[:,:,6]==1)
    #y[x[:,:,6] == 0] = y_offstroke_vector
    return y
#CELoss Test
x=np.zeros((1, 5, 7))
x[:,:,6]=[1,0,1,0,1]
y_pred=get_y_pred(x, y_offstroke_vector, y_onstroke_vector)
print(get_y_pred(x, y_offstroke_vector, y_onstroke_vector))

[[[0.0037037  0.0037037  0.0037037  0.0037037  0.0037037  0.0037037
   0.0037037  0.0037037  0.0037037  0.0037037  0.0037037  0.0037037
   0.0037037  0.0037037  0.0037037  0.0037037  0.0037037  0.0037037
   0.0037037  0.0037037  0.0037037  0.0037037  0.0037037  0.0037037
   0.0037037  0.0037037  0.0037037  0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.00873786 0.00873786 0.00873786
   0.00873786 0.00873786 0.00873786 0.

In [27]:
np.shape(y_pred)

(5, 122)

In [319]:
CELoss(y_pred, x)

NameError: name 'y_pred' is not defined

In [444]:
def MathModel(input_shape, output_dim, regularize=False):
    input = Input(shape=input_shape)
    X = Bidirectional(LSTM(128, return_sequences = True, kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4) if regularize else None))(input)
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences = True, kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4) if regularize else None))(X)
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences = True, kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4) if regularize else None))(X)
    X = Dropout(0.5)(X)
    X = Dense(output_dim+1, kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4) if regularize else None)(X)
    output = Activation('softmax')(X)
    
    model = Model(inputs=input,outputs=output)
    model.add_loss(CELoss(output,input))
    
    return model

In [402]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [403]:
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = tf.keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    # Iterate over the results and get back the text
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(index_to_value[result]).numpy().decode("utf-8")
        output_text.append(result)
    return output_text

In [404]:
import math
class MathDataset(tf.keras.utils.Sequence):
    def __init__(self, x, y, batch_size, X_train_data_len, Y_train_data_len):
        self.x=x
        self.y=y
        self.batch_size=batch_size
        self.X_train_data_len=X_train_data_len
        self.Y_train_data_len=Y_train_data_len
    def __len__(self):
        return math.floor(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        max_x=max(self.X_train_data_len[idx * self.batch_size:(idx + 1) *
        self.batch_size])
        max_y=max(self.Y_train_data_len[idx * self.batch_size:(idx + 1) *
        self.batch_size])
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size, :max_x]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size, :max_y]

        return (batch_x,batch_y)

In [416]:
np.shape(trainData.__getitem__(0)[1])

(32, 59)

In [409]:
def trainModel(model, trainData, evalData, learning_rate, momentum, epochs):
    #model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate,clipvalue=0.5, clipnorm=0.5), loss=CTCLoss)
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum), loss=CTCLoss)
    model.fit(x=trainData, validation_data=evalData, epochs=epochs)
    return model


In [359]:
type(Y_train[0][0])

int

In [406]:
import pickle as pkl
with open(datapath+"Train", "rb") as f:
    X_train, Y_train = pkl.load(f)
with open(datapath+"Test", "rb") as f:
    X_test, Y_test = pkl.load(f)
with open(datapath+"Eval", "rb") as f:
    X_eval, Y_eval = pkl.load(f)
NUM_FEATURES=7
len_train=len(X_train)
len_test=len(X_test)
len_eval=len(X_eval)
max_X_train_len = max([len(sample) for sample in X_train])
max_Y_train_len = max([len(sample) for sample in Y_train])
max_X_test_len = max([len(sample) for sample in X_test])
max_Y_test_len = max([len(sample) for sample in Y_test])
max_X_eval_len = max([len(sample) for sample in X_eval])
max_Y_eval_len = max([len(sample) for sample in Y_eval])
padding_vector=np.array([0,0,0,0,0,0,1])
eos_index=146
blank_index=147

X_train_data_len=[len(data) for data in X_train]
Y_train_data_len=[len(data) for data in Y_train]
X_test_data_len=[len(data) for data in X_test]
Y_test_data_len=[len(data) for data in Y_test]
X_eval_data_len=[len(data) for data in X_eval]
Y_eval_data_len=[len(data) for data in Y_eval]
padded_X_train=np.zeros((len_train, max_X_train_len, NUM_FEATURES), dtype=np.int32)+padding_vector
padded_Y_train=np.zeros((len_train, max_Y_train_len), dtype = np.int32)+eos_index
padded_X_test=np.zeros((len_test, max_X_test_len, NUM_FEATURES))+padding_vector
padded_Y_test=np.zeros((len_test, max_Y_test_len), dtype=np.int32)+eos_index
padded_X_eval=np.zeros((len_test, max_X_eval_len, NUM_FEATURES), dtype=np.int32)+padding_vector
padded_Y_eval=np.zeros((len_test, max_Y_eval_len), dtype=np.int32)+eos_index
for i in range(len_train):
    sample_X = np.array(X_train[i])
    padded_X_train[i,:len(sample_X),:]=sample_X
    sample_Y = np.array(Y_train[i])
    padded_Y_train[i,:len(sample_Y)]=sample_Y
        
for i in range(len_test):
    sample_X = np.array(X_test[i])
    padded_X_test[i,:len(sample_X),:]=sample_X
    sample_Y = np.array(Y_test[i])
    padded_Y_test[i,:len(sample_Y)]=sample_Y

for i in range(len_eval):
    sample_X = np.array(X_eval[i])
    padded_X_eval[i,:len(sample_X),:]=sample_X
    sample_Y = np.array(Y_eval[i])
    padded_Y_eval[i,:len(sample_Y)]=sample_Y
#trainData = MathDataset(padded_X_train[incorrectTrain], padded_Y_train[incorrectTrain], 32, list(np.array(X_train_data_len)[incorrectTrain]), list(np.array(Y_train_data_len)[incorrectTrain]))
trainData = MathDataset(padded_X_train, padded_Y_train, 32, X_train_data_len, Y_train_data_len)
testData = MathDataset(padded_X_test, padded_Y_test, 32, X_test_data_len, Y_test_data_len)
evalData = MathDataset(padded_X_eval, padded_Y_eval, 32, X_eval_data_len, Y_eval_data_len)
#evalData = MathDataset(padded_X_eval[incorrectEval], padded_Y_eval[incorrectEval], 32, list(np.array(X_eval_data_len)[incorrectEval]), list(np.array(Y_eval_data_len)[incorrectEval]))


In [410]:
model = MathModel(input_shape=(None,7), output_dim=vocab_size)

In [299]:
padded_X_eval[correctEval].shape

(855, 249, 7)

In [300]:
list(np.array(X_eval_data_len)[correctEval])

855

In [172]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.1), lossCLoss=CT)
inp = np.array([[[20.0, 75.0, 0.0, 0.0, 0.0, 0.0, 0], [30, 40, 5.0, 6.0, 1.0, -3.0, 1], [30, 40, 2.0, 4.0, -1.0, 2.0, 1], [30.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0], [25, -35.0, 1.0, 2.0, 1.0, -1.0, 1], [20.0, -50.0, 0.0, 0.0, 0.0, 0.0, 0]]], dtype=np.float32)
out = model(inp)
print(model.loss)

NameError: name 'CT' is not defined

In [281]:
correctTrain=[]
incorrectTrain=[]
for i, (lx, ly) in enumerate(zip(X_train_data_len, Y_train_data_len)):
    if lx > 2*ly:
        correctTrain.append(i)
    else:
        incorrectTrain.append(i)
correctEval=[]
incorrectEval=[]
for i, (lx, ly) in enumerate(zip(X_eval_data_len, Y_eval_data_len)):
    if lx > 2*ly:
        correctEval.append(i)
    else:
        incorrectEval.append(i)


In [233]:
len(incorrect)

2841

In [280]:
X_train_data_len=[len(data) for data in X_train]
Y_train_data_len=[len(data) for data in Y_train]
X_test_data_len=[len(data) for data in X_test]
Y_test_data_len=[len(data) for data in Y_test]
X_eval_data_len=[len(data) for data in X_eval]
Y_eval_data_len=[len(data) for data in Y_eval]

In [ ]:
#NOTE ADD NONSEG TO VOCAB POTENTIALLY FOR OFFSTROKES WITHIN A SYMBOL 
#Make the CE Loss weighted

In [282]:
padded_X_eval[correctEval].shape

(855, 249, 7)

In [347]:
trainData.y[0]

array([  0.,  63.,   0., 115.,   0.,  64.,   0., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 146., 146., 146., 146.,
       146., 146., 146., 146., 146., 146., 146., 14

In [448]:
#new_model=trainModel(model, trainData, evalData, 0.001, 0.9, 50)
full_model=trainModel(full_model, trainData, evalData, 0.0001, 0.9, 60)

Epoch 1/60
308/308 [==============================] - 1504s 5s/step - loss: 120.0202 - val_loss: 117.7017
Epoch 2/60
308/308 [==============================] - 1476s 5s/step - loss: 119.1277 - val_loss: 122.1248
Epoch 3/60
308/308 [==============================] - 1489s 5s/step - loss: 119.0541 - val_loss: 116.5453
Epoch 4/60
308/308 [==============================] - 1460s 5s/step - loss: 118.1297 - val_loss: 116.0984
Epoch 5/60
214/308 [===================>..........] - ETA: 7:04 - loss: 116.8564

In [312]:
for i in range(len(padded_X_eval[correctEval])):
    print(len(padded_X_eval[correctEval][i]), len(padded_Y_eval[correctEval][i]))

249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97
249 97

KeyboardInterrupt: 

In [425]:
with open("C:/Users/aiden/Downloads/ProjectData/index_to_symbol.pkl", "rb") as f:
    index_to_symbol = pkl.load(f)

In [446]:
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = tf.keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    batch_outputs=[]
    for result in results:
        output=[]
        for sym in result:
            sym=int(sym)
            if sym != -1 and sym != vocab_size-1:
                output.append(index_to_symbol[sym])
        batch_outputs.append(output)
    return batch_outputs

In [447]:
pred = new_model.predict(trainData.__getitem__(0)[0])
print(decode_batch_predictions(pred))

1/1 [==============================] - 4s 4s/step
[['Right', '0', 'Right', '\\lt', 'Right', 'x', 'Right', '\\lt', 'Right', '1'], ['Right', 'A', 'Right', '(', 'Right', 'B', 'Right', '+', 'Right', 'c', 'Right', ')', 'Right', '=', 'Right', 'A', 'Right', 'B', 'Right', '+', 'Right', 'A', 'Right', ')', 'Right'], ['Right', 'A', 'Right', '\\times', 'Right', '1', 'Right'], ['Right', 'A', 'Sub', '2', 'UFS', '\\gt', 'Right', 'B', 'Sub', '2', 'UFS'], ['Right', '\\forall', 'Right', 'x', 'Right', '\\in', 'Right', 'A', 'Right', 'COMMA', 'Right', '\\forall', 'Right', 'y', 'Right', '\\in', 'Right', 'B', 'Right', 'COMMA', 'Right', 'x', 'Right', 'Right'], ['Right', '\\forall', 'Right', 'a', 'Right', 'COMMA', 'Right', '\\forall', 'Right', '6', 'Right', 'COMMA', 'Right', '\\forall', 'Sub', '4', 'COMMA', 'Right', '(', 'Right', 'a', 'Sub', 'COMMA', 'UFS', '8', 'Right', 'COMMA', 'Right', 'c', 'Right', ')', 'Right', '=', 'Right', '(', 'Right', '(', 'Right', 'a', 'Right', 'COMMA', 'Right', '0', 'Right', ')', 'R

{'Right': 0,
 'Sub': 1,
 'Sup': 2,
 'Inside': 3,
 'UTF': 4,
 'NTB': 5,
 'BTD': 6,
 'UFD': 7,
 'DFS': 8,
 'UFS': 9,
 'OFI': 10,
 'DFS-NTB': 11,
 'LB': 12,
 'UFL': 13,
 'STS': 14,
 'DTI': 15,
 'ITL': 16,
 'DFL': 17,
 'UFS-NTB': 18,
 'DFS-UFD': 19,
 'UFS-UFD': 20,
 'DFS-DFS': 21,
 'OFI-NTB': 22,
 'OFI-UFD': 23,
 'DFS-OFI': 24,
 'OFI-OFI': 25,
 'UFD-DFS': 26,
 'Sup-UTF': 27,
 'UFS-UTF': 28,
 'UFL-UTF': 29,
 'Inside-UTF': 30,
 'UFD-OFI': 31,
 'DFS-OFI-UFD': 32,
 'NTB-UTF': 33,
 'UFS-UFS': 34,
 'DFL-UTF': 35,
 'UFD-UFD': 36,
 'UFS-OFI': 37,
 'UFS-DFS': 38,
 'Radical': 39,
 'RTI': 40,
 'RTI-UTF': 41,
 'Radical-UTF': 42,
 '!': 43,
 '(': 44,
 ')': 45,
 '+': 46,
 '-': 47,
 '.': 48,
 '/': 49,
 '0': 50,
 '1': 51,
 '2': 52,
 '3': 53,
 '4': 54,
 '5': 55,
 '6': 56,
 '7': 57,
 '8': 58,
 '9': 59,
 '<': 60,
 '=': 61,
 '>': 62,
 'A': 63,
 'B': 64,
 'C': 65,
 'COMMA': 66,
 'E': 67,
 'F': 68,
 'G': 69,
 'H': 70,
 'I': 71,
 'L': 72,
 'M': 73,
 'N': 74,
 'P': 75,
 'R': 76,
 'S': 77,
 'T': 78,
 'V': 79,
 'X':

In [419]:
new_model.summary()

Model: "model_46"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_49 (InputLayer)          [(None, None, 7)]    0           []                               
                                                                                                  
 bidirectional_140 (Bidirection  (None, None, 256)   139264      ['input_49[0][0]']               
 al)                                                                                              
                                                                                                  
 dropout_138 (Dropout)          (None, None, 256)    0           ['bidirectional_140[0][0]']      
                                                                                                  
 bidirectional_141 (Bidirection  (None, None, 256)   394240      ['dropout_138[0][0]']     